In [ ]:
import numpy as np
import random as rnd
import matplotlib.pyplot as plt

class Model():
    
    def __init__(self):
        self.weights = []
        self.biases = []
        # Layers will store the informations we need to create the layers
        # Layers will be created only when the feedforward begins because we need to know
        # the shape of the input layer, which is store as None for now
        self.layers_info = []
        self.n_layers = 0
        #self.layers_initiated = False

    def add_layer(self, size, activation):
        self.layers_info.append({"size": size,
                "activation": activation,
                "index": self.n_layers})
        self.n_layers += 1

    def initiate_layers(self, input_shape):
        # Create all the weights and biases of the model
        # Takes the input_shape in parameter because it needs to know how many input neurons there is to create the
        # appropriat number of weights and biases

        # Initiate the value of prev_layer_size to the size of the input layer
        prev_layer_size = 1
        for d in input_shape:
            prev_layer_size *= d

        # Creates all the other layer
        for l in range(len(self.layers_info)):
            # Takes the number of neurons in the l-1 th and in the lth layer to initiate the weights and biases in the
            # l th layer
            current_layer_size = self.layers_info[l]["size"]
            shape = [current_layer_size, prev_layer_size]
            self._create_layer(shape)

            # Change the value of prev_layer_size for the next iteration
            prev_layer_size = current_layer_size

    def predict(self, output, target):
        """prediction = np.argmax(outputs)
        if target[prediction] == 1:
            return True
        else:
            return False"""

        if output > 0.5:
            prediction = 1
        else:
            prediction = 0

        if prediction == target[0]:
            return True
        else:
            return False


    def predict_highest(self, output, target):
        prediction = np.argmax(outputs)
        if target[prediction] == 1:
            return True
        else:
            return False



    def fit(self, inputs, targets, epochs, eta=0.01, mini_batches_size=10):
        training_set = self.create_training_set(inputs, targets)
        for i in range(epochs):
            #tracking
            cost = 0
            accuracy = 0
            #tracking

            n_mini_batches = int( len(training_set)/mini_batches_size )
            print(n_mini_batches)
            rnd.shuffle(training_set)
            mini_batches = [training_set[k*mini_batches_size:(k+1)*mini_batches_size] for k in range(n_mini_batches)]

            for mini_batch in mini_batches:
                zs = []
                activations = []
                errors = []



                for training_input in mini_batch:
                    #feedforward
                    x = training_input[0]
                    t = [training_input[1]]
                    zs_x, activations_x = self._feedforward(x, training=True)
                    zs.append(zs_x)
                    activations.append(activations_x)

                    #backprop of the error
                    errors_x = self._backprop(zs_x, activations_x, t)
                    errors.append(errors_x)

                    #tracking
                    cost += self._cost(activations_x[-1], t)
                    #print("output: ", activations_x[-1])
                    prediction = self.predict_highest(activations_x[-1], t)
                    if prediction:
                        accuracy += 1
                    #tracking 

                for l in range(self.n_layers):
                    errors_act_mean = np.zeros(self.weights[l].shape)
                    errors_mean = np.zeros(self.biases[l].shape)

                    #print("errors_mean shape of layer {0}: {1}".format(l, errors_mean.shape))
                    # Computing the mean of all the terms errors*activation (or delta*a)
                    for error_x, activation_x in zip(errors, activations):
                        #print("error_x of layer {0}: \n{1}".format(l, error_x[l]))
                        #print("activations of layer {}: {}".format(l, activations_x[l]))
                        errors_act_mean += np.dot(error_x[l], activations_x[l].reshape(1, -1))
                        error_x[l] = error_x[l].reshape(-1,)
                        #print("error_x after reshaping", error_x[l])
                        errors_mean += error_x[l]
                    errors_act_mean = np.true_divide(errors_act_mean, mini_batches_size)
                    errors_mean = np.true_divide(errors_mean, mini_batches_size)

                    self.weights[l] -= eta * errors_act_mean
                    self.biases[l] -= eta * errors_mean

            #tracking
            cost = np.true_divide(cost, n_mini_batches)
            accuracy /= (n_mini_batches*mini_batches_size)
            print("\nepochs {}".format(i))
            print("Cost = {}".format(cost))
            print("Accuracy = {:.2f}%".format(accuracy*100))
            print("progression = {:.2f}%".format((100*(i+1)/epochs)))



    def _backprop(self, zs, activations, t):
        l = self.n_layers

        errors = []
        for layer in range(l):
            errors.append([])

        errors[-1] = self._cost_derivative(activations[-1], t) * self._activation_derivative(zs[-1], -1)

        for i in range(2, l+1):
            errors[l - i] = np.dot(self.weights[l+1 - i].transpose(), errors[l+1 - i]) * self._activation_derivative(zs[l - i], l-i)


        for layer in range(l):
            errors[layer] = errors[layer].reshape(-1, 1)

        return errors


    def summary(self):
        summary = "##################################\n"
        summary += "MODEL SUMMARY\n"
        summary += "Number of layers: {}\n".format(self.n_layers)
        summary  += "//////////////////////////////////\n"     
        for layer in self.layers_info:
            summary += "layer {}: ".format(layer["index"])
            summary += "Size: {} ".format(layer["size"])
            summary += "Activation: {}".format(layer["activation"])
            summary += "\n-------------------------------\n"
        summary += "End of summary\n"
        summary += "##################################\n"
        #print(summary)
        return(summary)

    def create_training_set(self, inputs, targets):

        training_set = []
        for x, t in zip(inputs, targets):
            training_set.append((x, t))

        return training_set


    def _feedforward(self, activation, training):
        # Flatten the input
        activation = self._flatten(activation)



        #If we call this function to train the network,
        # we need to record the pre-activations and activations
        if training:
            activations = [activation]
            zs = []

        for l in range(self.n_layers):

            # Compute pre_activation
            z = np.dot(self.weights[l], activation) + self.biases[l]
            # Compute activation
            activation = self._activation(z, l)

            if training:
                zs.append(z)
                activations.append(activation)




        if training:
            #print("activations:\n", activations)
            return zs, activations
        else:
            return activation


    def _activation(self, z, layer):
        act_fct = self.layers_info[layer]["activation"]
        if act_fct == "sigmoid":
            activation = self._sigmoid(z)
        elif act_fct == "relu":
            activation = self._relu(z)
        else:
            print("Error with the activation function of the layer number {}".format(layer))

        return activation

    def _activation_derivative(self, z, layer):
        act_fct = self.layers_info[layer]["activation"]
        if act_fct == "sigmoid":
            activation_prime = self._sigmoid_derivative(z)
        elif act_fct == "relu":
            activation_prime = self._relu_derivative(z)
        else:
            print("Error with the activation function of the layer number {}".format(layer))

        return activation_prime

    def _create_layer(self, shape):
        self.weights.append(np.random.randn(*shape))
        self.biases.append(np.zeros(shape[0]))
        #self.biases[-1] = self.biases[-1].reshape(-1, 1)


    def _relu(self, z):
        shape = z.shape
        z = self._flatten(z)
        new_z = []
        for element in z:
            if element <= 0:
                new_z.append(0)
            else:
                new_z.append(element)
        new_z = np.array(new_z)
        return new_z.reshape(shape)

    def _sigmoid(self, z):
        return 1 / (1 + np.e ** (-z))


    def _flatten(self, vector):
        shape = vector.shape
        size = 1
        for dimension in shape:
            size *= dimension

        return vector.reshape(size)

    def _cost(self, y, t):
        return 0.5*(y - t)**2

    def _cost_derivative(self, y, t):
        return (y - t)

    def _sigmoid_derivative(self, z):
        return self._sigmoid(z) * (1 - self._sigmoid(z))

    def _relu_derivative(self, z):
        shape = z.shape
        z = self._flatten(z)
        new_z = []
        for element in z:
            if element <= 0:
                new_z.append(0)
            else:
                new_z.append(1)
        new_z = np.array(new_z)
        return new_z.reshape(shape)

In [ ]:
model =Model()

In [ ]:
def get_dataset(row_per_class):
    """
    Method used to get the dataset
    """



    #generate rows

    sick = np.random.randn(row_per_class, 2) + np.array([-2, 2])
    sick2 = np.random.randn(row_per_class, 2) + np.array([2, -2])


    healthy = np.random.randn(row_per_class, 2) + np.array([2, 2])
    healthy2 = np.random.randn(row_per_class, 2) + np.array([-2, -2])

    features = np.vstack([sick, sick2, healthy, healthy2])
    targets = np.concatenate((np.zeros(row_per_class*2) + 1, np.zeros(row_per_class*2)))
    return features, targets

In [ ]:
features, targets = get_dataset(10)
plt.scatter(features[0:21].transpose()[0], features[0:21].transpose()[1], c="blue")
plt.scatter(features[21:].transpose()[0], features[21:].transpose()[1], c="red")
plt.show()
print(features.shape)
print(features[0:4])
print(targets[0:4])

In [ ]:
model.add_layer(10, "relu")
model.add_layer(5, "relu")
model.add_layer(1, "sigmoid")
print(model.summary())

In [ ]:
model.initiate_layers([2])

In [ ]:
model._feedforward(features[4], targets[4])

In [ ]:
model.fit(features, targets, 200)

In [ ]:
print("features: ", features)
print("targets: ", targets)
for feature in features:
    prediction = model._feedforward(feature, False)
    print(prediction)